# Trading Strategy Optimization
Run this notebook in Google Colab for free cloud computing with visualizations.

In [ ]:
# Install dependencies (yfinance works from any location, no geo-restrictions)
!pip install yfinance pandas numpy matplotlib seaborn plotly tqdm -q

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from itertools import product
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("Libraries loaded!")

In [ ]:
# Configuration - Using Yahoo Finance tickers (no geo-restrictions!)
SYMBOLS = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'SOL-USD', 'ADA-USD']

# Parameters to optimize
STOP_LOSSES = [0.01, 0.015, 0.02, 0.025, 0.03]      # 1% - 3%
TAKE_PROFITS = [0.02, 0.03, 0.04, 0.05, 0.06]      # 2% - 6%
TIME_EXITS = [1, 2, 4, 8, 12, 24, None]            # hours
SIGNAL_THRESHOLDS = [0.1, 0.15, 0.2, 0.25]         # entry threshold

total_combinations = len(STOP_LOSSES) * len(TAKE_PROFITS) * len(TIME_EXITS) * len(SIGNAL_THRESHOLDS)
print(f"Total combinations to test: {total_combinations}")

In [ ]:
# Fetch historical data using Yahoo Finance (works globally, no restrictions!)
def fetch_candles(symbol, period='90d', interval='1h'):
    """Fetch historical candles from Yahoo Finance"""
    print(f"Fetching {symbol}...")
    
    ticker = yf.Ticker(symbol)
    df = ticker.history(period=period, interval=interval)
    
    if df.empty:
        print(f"  Warning: No data for {symbol}")
        return None
    
    # Rename columns to match our format
    df = df.rename(columns={
        'Open': 'open',
        'High': 'high', 
        'Low': 'low',
        'Close': 'close',
        'Volume': 'volume'
    })
    
    print(f"  Got {len(df)} candles")
    return df[['open', 'high', 'low', 'close', 'volume']]

# Load data for all symbols
all_data = {}
for symbol in SYMBOLS:
    df = fetch_candles(symbol, period='90d', interval='1h')
    if df is not None and len(df) > 100:
        all_data[symbol] = df

print(f"\nLoaded {len(all_data)} symbols")
print(f"Total candles: {sum(len(df) for df in all_data.values())}")

In [ ]:
def calculate_signal(closes, volumes):
    """Calculate trading signal"""
    if len(closes) < 50:
        return 0.0
    
    score = 0.0
    
    # RSI
    deltas = np.diff(closes[-15:])
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    avg_gain = np.mean(gains)
    avg_loss = np.mean(losses)
    
    if avg_loss == 0:
        rsi = 100 if avg_gain > 0 else 50
    else:
        rsi = 100 - (100 / (1 + avg_gain / avg_loss))
    
    if rsi < 30:
        score += 0.3
    elif rsi > 70:
        score -= 0.3
    elif rsi < 40:
        score += 0.1
    elif rsi > 60:
        score -= 0.1
    
    # EMA trend
    ema_9 = np.mean(closes[-9:])
    ema_21 = np.mean(closes[-21:])
    ema_50 = np.mean(closes[-50:])
    
    if ema_9 > ema_21 > ema_50:
        score += 0.2
    elif ema_9 < ema_21 < ema_50:
        score -= 0.2
    
    # Volume
    avg_vol = np.mean(volumes[-20:])
    if volumes[-1] > avg_vol * 1.5:
        if score > 0:
            score += 0.1
        elif score < 0:
            score -= 0.1
    
    # Momentum
    momentum = (closes[-1] - closes[-10]) / closes[-10]
    if momentum > 0.02:
        score += 0.15
    elif momentum < -0.02:
        score -= 0.15
    
    return max(-1.0, min(1.0, score))

In [ ]:
def backtest_params(df, stop_loss, take_profit, time_exit, signal_threshold):
    """Backtest one parameter combination"""
    capital = 10000.0
    position = None
    trades = []
    
    closes = df['close'].values
    highs = df['high'].values
    lows = df['low'].values
    volumes = df['volume'].values
    timestamps = df.index
    
    for i in range(50, len(df)):
        price = closes[i]
        high = highs[i]
        low = lows[i]
        current_time = timestamps[i]
        
        score = calculate_signal(closes[:i+1], volumes[:i+1])
        
        if position:
            entry_price = position['entry_price']
            side = position['side']
            entry_time = position['entry_time']
            
            if side == 'LONG':
                pnl_pct = (price - entry_price) / entry_price
                hit_sl = low <= entry_price * (1 - stop_loss)
                hit_tp = high >= entry_price * (1 + take_profit)
            else:
                pnl_pct = (entry_price - price) / entry_price
                hit_sl = high >= entry_price * (1 + stop_loss)
                hit_tp = low <= entry_price * (1 - take_profit)
            
            hours_in_position = (current_time - entry_time).total_seconds() / 3600
            
            exit_reason = None
            exit_price = price
            
            if hit_sl:
                exit_reason = 'SL'
                exit_price = entry_price * (1 - stop_loss) if side == 'LONG' else entry_price * (1 + stop_loss)
            elif hit_tp:
                exit_reason = 'TP'
                exit_price = entry_price * (1 + take_profit) if side == 'LONG' else entry_price * (1 - take_profit)
            elif time_exit and hours_in_position >= time_exit:
                exit_reason = 'TIME'
            elif side == 'LONG' and score < -signal_threshold:
                exit_reason = 'SIGNAL'
            elif side == 'SHORT' and score > signal_threshold:
                exit_reason = 'SIGNAL'
            
            if exit_reason:
                if side == 'LONG':
                    pnl = (exit_price - entry_price) / entry_price * position['size']
                else:
                    pnl = (entry_price - exit_price) / entry_price * position['size']
                
                pnl -= position['size'] * 0.002  # commission
                capital += pnl
                
                trades.append({
                    'side': side,
                    'entry': entry_price,
                    'exit': exit_price,
                    'pnl': pnl,
                    'reason': exit_reason,
                    'hours': hours_in_position
                })
                position = None
        
        if not position:
            if score > signal_threshold:
                position = {
                    'side': 'LONG',
                    'entry_price': price,
                    'entry_time': current_time,
                    'size': capital * 0.02
                }
            elif score < -signal_threshold:
                position = {
                    'side': 'SHORT',
                    'entry_price': price,
                    'entry_time': current_time,
                    'size': capital * 0.02
                }
    
    if not trades:
        return None
    
    wins = [t for t in trades if t['pnl'] > 0]
    losses_trades = [t for t in trades if t['pnl'] <= 0]
    
    total_pnl = sum(t['pnl'] for t in trades)
    win_rate = len(wins) / len(trades) if trades else 0
    
    wins_sum = sum(t['pnl'] for t in wins)
    losses_sum = abs(sum(t['pnl'] for t in losses_trades))
    profit_factor = wins_sum / losses_sum if losses_sum > 0 else 0
    
    return {
        'total_pnl': total_pnl,
        'return_pct': (capital - 10000) / 10000 * 100,
        'trades': len(trades),
        'win_rate': win_rate,
        'profit_factor': profit_factor,
        'final_capital': capital
    }

In [ ]:
# Run optimization
results = []

combinations = list(product(STOP_LOSSES, TAKE_PROFITS, TIME_EXITS, SIGNAL_THRESHOLDS))

for sl, tp, time_exit, threshold in tqdm(combinations, desc="Optimizing"):
    total_return = 0
    total_trades = 0
    total_wins = 0
    symbol_results = []
    
    for symbol, df in all_data.items():
        result = backtest_params(df, sl, tp, time_exit, threshold)
        if result:
            total_return += result['return_pct']
            total_trades += result['trades']
            total_wins += result['trades'] * result['win_rate']
            symbol_results.append(result)
    
    if symbol_results:
        avg_return = total_return / len(symbol_results)
        avg_win_rate = total_wins / total_trades if total_trades > 0 else 0
        
        results.append({
            'stop_loss': sl,
            'take_profit': tp,
            'time_exit': time_exit,
            'threshold': threshold,
            'avg_return': avg_return,
            'total_trades': total_trades,
            'win_rate': avg_win_rate,
            'profit_factor': np.mean([r['profit_factor'] for r in symbol_results])
        })

# Sort by return
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('avg_return', ascending=False)

print(f"\nOptimization complete! {len(results)} combinations tested.")

In [ ]:
# TOP 10 Results
print("=" * 70)
print("TOP 10 BEST PARAMETER COMBINATIONS")
print("=" * 70)

for i, row in results_df.head(10).iterrows():
    time_str = f"{row['time_exit']}h" if row['time_exit'] else "No limit"
    print(f"\nReturn: {row['avg_return']:+.2f}%")
    print(f"  SL: {row['stop_loss']*100:.1f}% | TP: {row['take_profit']*100:.1f}% | Time: {time_str} | Threshold: {row['threshold']}")
    print(f"  Trades: {row['total_trades']} | Win Rate: {row['win_rate']:.1%} | PF: {row['profit_factor']:.2f}")

In [ ]:
# VISUALIZATION 1: Heatmap of SL vs TP (averaged across other params)
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Return heatmap
pivot_return = results_df.groupby(['stop_loss', 'take_profit'])['avg_return'].mean().unstack()
sns.heatmap(pivot_return, annot=True, fmt='.1f', cmap='RdYlGn', center=0, ax=axes[0])
axes[0].set_title('Average Return (%) by Stop Loss vs Take Profit')
axes[0].set_xlabel('Take Profit')
axes[0].set_ylabel('Stop Loss')

# Win rate heatmap
pivot_winrate = results_df.groupby(['stop_loss', 'take_profit'])['win_rate'].mean().unstack()
sns.heatmap(pivot_winrate, annot=True, fmt='.1%', cmap='RdYlGn', ax=axes[1])
axes[1].set_title('Win Rate by Stop Loss vs Take Profit')
axes[1].set_xlabel('Take Profit')
axes[1].set_ylabel('Stop Loss')

plt.tight_layout()
plt.savefig('optimization_heatmap.png', dpi=150)
plt.show()

In [ ]:
# VISUALIZATION 2: Time Exit analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

time_analysis = results_df.groupby('time_exit').agg({
    'avg_return': 'mean',
    'win_rate': 'mean',
    'total_trades': 'mean'
}).reset_index()

time_labels = [str(t) + 'h' if t else 'No limit' for t in time_analysis['time_exit']]

axes[0].bar(time_labels, time_analysis['avg_return'], color=['green' if x > 0 else 'red' for x in time_analysis['avg_return']])
axes[0].set_title('Average Return by Time Exit')
axes[0].set_xlabel('Time Exit')
axes[0].set_ylabel('Return (%)')
axes[0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)

axes[1].bar(time_labels, time_analysis['win_rate'] * 100, color='steelblue')
axes[1].set_title('Win Rate by Time Exit')
axes[1].set_xlabel('Time Exit')
axes[1].set_ylabel('Win Rate (%)')

plt.tight_layout()
plt.savefig('optimization_time_exit.png', dpi=150)
plt.show()

In [ ]:
# VISUALIZATION 3: Signal Threshold analysis
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

threshold_analysis = results_df.groupby('threshold').agg({
    'avg_return': 'mean',
    'win_rate': 'mean',
    'total_trades': 'mean'
}).reset_index()

axes[0].bar(threshold_analysis['threshold'].astype(str), threshold_analysis['avg_return'], 
            color=['green' if x > 0 else 'red' for x in threshold_analysis['avg_return']])
axes[0].set_title('Return by Signal Threshold')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Return (%)')

axes[1].bar(threshold_analysis['threshold'].astype(str), threshold_analysis['win_rate'] * 100, color='steelblue')
axes[1].set_title('Win Rate by Threshold')
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('Win Rate (%)')

axes[2].bar(threshold_analysis['threshold'].astype(str), threshold_analysis['total_trades'], color='orange')
axes[2].set_title('Avg Trades by Threshold')
axes[2].set_xlabel('Threshold')
axes[2].set_ylabel('Number of Trades')

plt.tight_layout()
plt.savefig('optimization_threshold.png', dpi=150)
plt.show()

In [ ]:
# VISUALIZATION 4: Interactive 3D scatter plot
fig = px.scatter_3d(
    results_df,
    x='stop_loss',
    y='take_profit',
    z='avg_return',
    color='avg_return',
    size='total_trades',
    hover_data=['time_exit', 'threshold', 'win_rate'],
    color_continuous_scale='RdYlGn',
    title='Optimization Results: Stop Loss vs Take Profit vs Return'
)
fig.show()

In [ ]:
# VISUALIZATION 5: Distribution of results
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].hist(results_df['avg_return'], bins=30, color='steelblue', edgecolor='black')
axes[0, 0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[0, 0].axvline(x=results_df['avg_return'].iloc[0], color='green', linestyle='--', linewidth=2, label='Best')
axes[0, 0].set_title('Distribution of Returns')
axes[0, 0].set_xlabel('Return (%)')
axes[0, 0].legend()

axes[0, 1].hist(results_df['win_rate'], bins=20, color='orange', edgecolor='black')
axes[0, 1].set_title('Distribution of Win Rates')
axes[0, 1].set_xlabel('Win Rate')

axes[1, 0].scatter(results_df['total_trades'], results_df['avg_return'], alpha=0.5)
axes[1, 0].set_title('Trades vs Return')
axes[1, 0].set_xlabel('Number of Trades')
axes[1, 0].set_ylabel('Return (%)')

axes[1, 1].scatter(results_df['win_rate'], results_df['avg_return'], alpha=0.5, c=results_df['profit_factor'], cmap='viridis')
axes[1, 1].set_title('Win Rate vs Return (color = Profit Factor)')
axes[1, 1].set_xlabel('Win Rate')
axes[1, 1].set_ylabel('Return (%)')

plt.tight_layout()
plt.savefig('optimization_distributions.png', dpi=150)
plt.show()

In [ ]:
# BEST PARAMETERS - Copy these to your trading bot!
best = results_df.iloc[0]

print("=" * 70)
print("OPTIMAL PARAMETERS FOR YOUR TRADING BOT")
print("=" * 70)
print(f"""
Copy these values to your auto_trading configuration:

STOP_LOSS = {best['stop_loss']}      # {best['stop_loss']*100:.1f}%
TAKE_PROFIT = {best['take_profit']}    # {best['take_profit']*100:.1f}%
TIME_EXIT = {best['time_exit']}           # hours (None = no limit)
SIGNAL_THRESHOLD = {best['threshold']}   # entry threshold

Expected performance (backtest):
  - Average Return: {best['avg_return']:+.2f}%
  - Win Rate: {best['win_rate']:.1%}
  - Profit Factor: {best['profit_factor']:.2f}
  - Total Trades: {best['total_trades']}
""")

In [ ]:
# Save results to CSV for further analysis
results_df.to_csv('optimization_results.csv', index=False)
print("Results saved to optimization_results.csv")

# Download files (in Colab)
try:
    from google.colab import files
    files.download('optimization_results.csv')
    files.download('optimization_heatmap.png')
    files.download('optimization_distributions.png')
except:
    print("Not running in Colab - files saved locally")